In [5]:
import boto3
import sagemaker
from sagemaker import get_execution_role

# Set custom bucket
bucket = 'insurance-prediction-bucket'  # Your custom S3 bucket
prefix = 'insurance-data'
role = get_execution_role()

# Local file to upload
local_file = 'insurance_pre.csv'

# Upload file to your specific bucket
session = sagemaker.Session()
s3_path = session.upload_data(path=local_file, bucket=bucket, key_prefix=prefix)

from utils import append_to_file
append_to_file("output.txt","s3-path:",s3_path)


print(f"✅ File uploaded to your bucket at: {s3_path}")


✅ File uploaded to your bucket at: s3://insurance-prediction-bucket/insurance-data/insurance_pre.csv


In [2]:
from sagemaker.sklearn.estimator import SKLearn
from sagemaker import get_execution_role
import sagemaker
from utils import append_to_file
session = sagemaker.Session()
role = get_execution_role()
bucket = 'insurance-prediction-bucket'  # Your custom S3 bucket
prefix = 'insurance-data'
sklearn_estimator = SKLearn(
    entry_point="train.py",
    role=role,
    instance_type="ml.m5.large",  # or "ml.t2.medium" for free tier
    framework_version="1.2-1",
    py_version="py3",
    output_path=f"s3://{bucket}/{prefix}/output",
    sagemaker_session=session
)
append_to_file("output.txt","framework_version 1.2-1")
# Start training
sklearn_estimator.fit({"train": f"s3://insurance-prediction-bucket/insurance-data/"})

image_uri=sklearn_estimator.training_image_uri()

append_to_file("output.txt","Image URI",image_uri)

model_s3_uri = sklearn_estimator.model_data

append_to_file("output.txt","model_s3_uri",model_s3_uri)

print("Model S3 URI:", model_s3_uri)





[05/02/25 15:20:40] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=715758;file:///opt/conda/lib/python3.12/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=710504;file:///opt/conda/lib/python3.12/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name:                                       ]8;id=60888;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=622469;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py#1042\1042]8;;\
                             sagemaker-scikit-learn-2025-05-02-15-20-40-817                                        

2025-05-02 15:20:41 Starting - Starting the training job......
2025-05-02 15:21:44 Downloading - Downloading input data...
2025-05-02 15:22:09 Downloading - Downloading the training image......
2025-05-02 15:23:15 Training - Training image download completed. Training in progress..2025-05-02 15:23:16,405 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2025-05-02 15:23:16,409 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-05-02 15:23:16,412 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2025-05-02 15:23:16,429 sagemaker_sklearn_container.training INFO     Invoking user training script.
2025-05-02 15:23:16,627 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-05-02 15:23:16,630 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2025-05-02 15:23:16,649 sagemaker-training-toolkit INFO     No GPUs

In [21]:
from sagemaker.sklearn.model import SKLearnModel
model = SKLearnModel(
    model_data="s3://insurance-prediction-bucket/insurance-data/output/sagemaker-scikit-learn-2025-05-02-15-20-40-817/output/model.tar.gz",  # <- from training
    role=role,
    entry_point="inference.py",                         # ✅ NEW inference script
    framework_version="1.2-1",
    py_version="py3",
    sagemaker_session=session
)

In [22]:
predictor = model.deploy(
    instance_type="ml.m5.large",
    initial_instance_count=1,
    endpoint_name="insurance-charge-endpoint4"
)

[05/02/25 16:08:27] INFO     Creating model with name:                                              ]8;id=323997;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=812358;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py#4094\4094]8;;\
                             sagemaker-scikit-learn-2025-05-02-16-08-27-065                                        

                    INFO     Creating endpoint-config with name insurance-charge-endpoint4          ]8;id=368709;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=787390;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py#5937\5937]8;;\

[05/02/25 16:08:28] INFO     Creating endpoint with name insurance-charge-endpoint4                 ]8;id=287611;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=548148;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py#4759\4759]8;;\

---------!

In [27]:
def predict_insurance_charge(predictor, age, bmi, children, sex_male, smoker_yes):
    """
    Sends a prediction request to the SageMaker endpoint.

    Parameters:
    - predictor: The SageMaker Predictor object.
    - age, bmi, children: Numeric inputs
    - sex_male, smoker_yes: Binary categorical encodings (0 or 1)
    - region_*: One-hot region encodings (only one should be 1)

    Returns:
    - Predicted insurance charge (float)
    """
    import json
    from sagemaker.serializers import JSONSerializer
    from sagemaker.deserializers import JSONDeserializer

    predictor.serializer = JSONSerializer()
    predictor.deserializer = JSONDeserializer()
    
    input_data = {
        "age": [age],
        "bmi": [bmi],
        "children": [children],
        "sex_male": [sex_male],
        "smoker_yes": [smoker_yes],
    }

    #input_date=json.dumps(input_data)
    #print(input_date)
    prediction = predictor.predict(input_data)
    return prediction


In [28]:
result = predict_insurance_charge(
    predictor,
    age=35,
    bmi=28.4,
    children=2,
    sex_male=1,
    smoker_yes=0
)

print("Predicted charge:", result)


Predicted charge: [6780.8349163]


In [37]:
import boto3

client = boto3.client('sagemaker', region_name='us-east-1')
response = client.list_endpoints()
for ep in response['Endpoints']:
    print(ep['EndpointName'])


insurance-charge-serverless-endpoint


In [38]:
from sagemaker import Predictor

# Use the same credentials/session setup as before
predictor = Predictor(endpoint_name="insurance-charge-serverless-endpoint")
predictor.delete_endpoint()
print("✅ Endpoint deleted successfully.")


[05/02/25 18:07:29] INFO     Deleting endpoint configuration with name:                             ]8;id=936117;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=505301;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py#4913\4913]8;;\
                             insurance-charge-serverless-endpoint                                                  

                    INFO     Deleting endpoint with name: insurance-charge-serverless-endpoint      ]8;id=253060;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=122038;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py#4903\4903]8;;\

✅ Endpoint deleted successfully.


In [ ]:
# Serverless

In [33]:
from sagemaker.sklearn.model import SKLearnModel
from sagemaker import get_execution_role, Session

session = Session()
role = get_execution_role()

model = SKLearnModel(
    model_data="s3://insurance-prediction-bucket/insurance-data/output/sagemaker-scikit-learn-2025-05-02-15-20-40-817/output/model.tar.gz",
    role=role,
    entry_point="inference.py",
    framework_version="1.2-1",
    py_version="py3",
    sagemaker_session=session
)

predictor = model.deploy(
    endpoint_name="insurance-charge-serverless-endpoint",
    serverless_inference_config={
        "MemorySizeInMB": 1024,
        "MaxConcurrency": 5
    }
)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:16                                                                                   │
│                                                                                                  │
│   13 │   sagemaker_session=session                                                               │
│   14 )                                                                                           │
│   15                                                                                             │
│ ❱ 16 predictor = model.deploy(                                                                   │
│   17 │   endpoint_name="insurance-charge-serverless-endpoint",                                   │
│   18 │   serverless_inference_config={                                                           │
│   19 │   │   "MemorySizeInMB": 1024,                                                             │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/model.py:1585 in deploy                        │
│                                                                                                  │
│   1582 │   │   │   )                                                                             │
│   1583 │   │                                                                                     │
│   1584 │   │   if is_serverless and not isinstance(serverless_inference_config, ServerlessInfer  │
│ ❱ 1585 │   │   │   raise ValueError(                                                             │
│   1586 │   │   │   │   "serverless_inference_config needs to be a ServerlessInferenceConfig obj  │
│   1587 │   │   │   )                                                                             │
│   1588                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: serverless_inference_config needs to be a ServerlessInferenceConfig object

In [34]:
from sagemaker.serverless import ServerlessInferenceConfig

model = SKLearnModel(
    model_data="s3://insurance-prediction-bucket/insurance-data/output/sagemaker-scikit-learn-2025-05-02-15-20-40-817/output/model.tar.gz",
    role=role,
    entry_point="inference.py",
    framework_version="1.2-1",
    py_version="py3",
    sagemaker_session=session
)

predictor = model.deploy(
    endpoint_name="insurance-charge-serverless-endpoint",
    serverless_inference_config=ServerlessInferenceConfig(
        memory_size_in_mb=1024,
        max_concurrency=5
    )
)


[05/02/25 17:42:37] INFO     Defaulting to only supported image scope: cpu.                       ]8;id=50145;file:///opt/conda/lib/python3.12/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=261611;file:///opt/conda/lib/python3.12/site-packages/sagemaker/image_uris.py#534\534]8;;\

                    INFO     Creating model with name:                                              ]8;id=441884;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=43294;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py#4094\4094]8;;\
                             sagemaker-scikit-learn-2025-05-02-17-42-37-576                                        

[05/02/25 17:42:38] INFO     Creating endpoint-config with name                                     ]8;id=156001;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=457921;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py#5937\5937]8;;\
                             insurance-charge-serverless-endpoint                                                  

                    INFO     Creating endpoint with name insurance-charge-serverless-endpoint       ]8;id=805402;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=925561;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py#4759\4759]8;;\

-----------------------------------------!

In [35]:
def predict_insurance_charge(predictor, age, bmi, children, sex_male, smoker_yes):
    """
    Sends a prediction request to the SageMaker endpoint.

    Parameters:
    - predictor: The SageMaker Predictor object.
    - age, bmi, children: Numeric inputs
    - sex_male, smoker_yes: Binary categorical encodings (0 or 1)
    - region_*: One-hot region encodings (only one should be 1)

    Returns:
    - Predicted insurance charge (float)
    """
    import json
    from sagemaker.serializers import JSONSerializer
    from sagemaker.deserializers import JSONDeserializer

    predictor.serializer = JSONSerializer()
    predictor.deserializer = JSONDeserializer()
    
    input_data = {
        "age": [age],
        "bmi": [bmi],
        "children": [children],
        "sex_male": [sex_male],
        "smoker_yes": [smoker_yes],
    }

    #input_date=json.dumps(input_data)
    #print(input_date)
    prediction = predictor.predict(input_data)
    return prediction


In [36]:
result = predict_insurance_charge(
    predictor,
    age=35,
    bmi=28.4,
    children=2,
    sex_male=1,
    smoker_yes=0
)

print("Predicted charge:", result)


Predicted charge: [6780.8349163]
